# Assignment #3: A simple language classifier with scikit-learn and PyTorch

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired and simplified from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks with sklearn and PyTorch
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [1]:
import bz2
import json
import os
import numpy as np
import requests
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import hashlib
from tqdm import tqdm
import pickle


In [2]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

## Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider six languages only: French (fra), Japanese (jpn), Chinese (cmn), English (eng), Swedish (swe), and Danish (dan). Below is an excerpt of the Tatoeba dataset limited to three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 23, 2021.

### Understanding the ${X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified ${X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will __include this matrix in your report__. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

To help you check your counts, you can use the `str.count()` function

In [3]:
example_ngrams = ['a', 'b', 'n', 'an', 'ba', 'na']

In [4]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('a')

8

In [5]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('an')

1

In [6]:
my_string = """Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."""
row = []
for ngram in example_ngrams:
    row += [my_string.count(ngram)]
row

[8, 0, 8, 1, 0, 0]

## Getting the Dataset

Before you start programming, download the Tatoeba dataset. You can use the instructions:

In [7]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2023-09-25 13:39:00--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183539203 (175M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2’

sentences.tar.bz2     3%[                    ]   5.80M  14.5MB/s               

sentences.tar.bz2   100%[===================>] 175.04M  23.6MB/s    in 7.6s    

2023-09-25 13:39:08 (23.1 MB/s) - ‘sentences.tar.bz2’ saved [183539203/183539203]



In [8]:
!tar -xvjf sentences.tar.bz2

sentences.csv


### Loading the Dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [9]:
dataset_large = open('sentences.csv', encoding='utf8').read().strip()
dataset_large = dataset_large.split('\n')
dataset_large[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

The size may vary as new documents are added every day to _Tatoeba_

In [10]:
len(dataset_large)

11626127

Run the code to split the fields and remove possible whitespaces

In [11]:
dataset_large = list(map(lambda x: tuple(x.split('\t')), dataset_large))
dataset_large = list(map(lambda x: tuple(map(str.strip, x)), dataset_large))
dataset_large[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

In [12]:
from collections import Counter
counter = Counter(map(lambda x: x[1], dataset_large))

Again the figures may vary

In [13]:
counter.most_common(30)

[('eng', 1832557),
 ('rus', 1015379),
 ('ita', 863210),
 ('tur', 729325),
 ('epo', 728733),
 ('kab', 686765),
 ('ber', 650115),
 ('deu', 627037),
 ('fra', 569043),
 ('por', 422236),
 ('spa', 402610),
 ('hun', 394834),
 ('jpn', 238601),
 ('heb', 200845),
 ('ukr', 184423),
 ('nld', 177097),
 ('fin', 146735),
 ('pol', 123050),
 ('lit', 93949),
 ('mkd', 78095),
 ('tgl', 74924),
 ('ces', 73650),
 ('cmn', 73646),
 ('mar', 73277),
 ('ara', 62803),
 ('dan', 59011),
 ('tok', 54067),
 ('swe', 53199),
 ('lat', 49211),
 ('srp', 47878)]

## Restricting the Dataset to a few Languages 

The Tatoeba dataset is very large. You will first extract a subset of it

Write the code to extract texts in the languages below. For each language, you limit the number of documents to 50,000 or less if the language has less documents.
You will call the resulting dataset: `dataset`

The languages

In [14]:
langs = ['fra', 'cmn', 'jpn', 'eng', 'swe', 'dan']

The maximal number of documents per language

In [15]:
MAX_DOCS = 50000

Write a loop that:
1. Extracts a list of all the documents in a certain language from the dataset
2. Shuffles this list with `random.shuffle()`
3. Adds `MAX_DOCS` to `dataset`. You just need to use a slice

In [16]:
# Write your code here
dataset = []
for lang in langs:
    dataset += list(filter(lambda x: x[1] == lang, dataset_large))[:MAX_DOCS]

In [17]:
random.shuffle(dataset)

In [18]:
len(dataset)

300000

In [19]:
dataset[:5]

[('8638547', 'dan', 'Min far er for nylig vendt tilbage fra Japan.'),
 ('98775', 'jpn', '彼らがいつやってくるかわからない。'),
 ('84287', 'jpn', '腐れ縁と思ってあきらめる。'),
 ('81442', 'jpn', '魔女の女王は死ぬ。'),
 ('611992', 'cmn', '香橙含有豐富的維他命 C。')]

### Pickel dump and reading the dataset

In [20]:
pickle.dump(dataset, open("lab4dataset.txt", "wb"))

In [21]:
dataset = pickle.load(open("lab4dataset.txt", "rb"))
dataset[:5]

[('8638547', 'dan', 'Min far er for nylig vendt tilbage fra Japan.'),
 ('98775', 'jpn', '彼らがいつやってくるかわからない。'),
 ('84287', 'jpn', '腐れ縁と思ってあきらめる。'),
 ('81442', 'jpn', '魔女の女王は死ぬ。'),
 ('611992', 'cmn', '香橙含有豐富的維他命 C。')]

## Utilities

Before you can use the dataset to train a model, you need to convert it into numbers. You will carry this with out the following steps and you will write a corresponding function.
1. You will extract the $n$-grams up to trigrams (`all_ngrams()`);
2. Trigrams can create many symbols that most student's machines cannot process. You will reduce their numbers using hash codes (`hash_ngrams()`);
3. You will compute the relative frequencies of the $n$-grams, replaced here by the hash codes (`calc_ref_freq()`).
4. The results will be stored in three dictionaries, for characters, bigrams, and trigrams. You will merge these dictionaries into one (`shift_keys()`).

You will then apply the functions to vectorize the dataset.

### Extracting $n$-grams
The goal of this section is that you extract the $n$-grams from a text. By default, you will lowercase the text. The result will have the form: `[chars, bigrams, trigrams]`

Write a function to extract the $n$-grams of a sentence: `ngrams(sentence, n=1, lc=True)`, `n` is a parameters. You can use list slices for this.

In [22]:
# Write your code here
def ngrams(sentence, n=1, lc=True):
    ngram_l = []
    chars = list(sentence)
    for i in range(len(chars)-n+1):
        ngram = "".join(chars[i:i+n])
        if lc:
            ngram = ngram.lower()
        ngram_l.append(ngram)
    return ngram_l

In [23]:
ngrams('try something.')

['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.']

In [24]:
ngrams('try something.', n=2)

['tr', 'ry', 'y ', ' s', 'so', 'om', 'me', 'et', 'th', 'hi', 'in', 'ng', 'g.']

We now use this function to extract all the $n$-grams

In [25]:
def all_ngrams(sentence, max_ngram=3, lc=True):
    all_ngram_list = []
    for i in range(1, max_ngram + 1):
        all_ngram_list += [ngrams(sentence, n=i, lc=lc)]
    return all_ngram_list

In [26]:
all_ngrams('try something.')

[['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.'],
 ['tr',
  'ry',
  'y ',
  ' s',
  'so',
  'om',
  'me',
  'et',
  'th',
  'hi',
  'in',
  'ng',
  'g.'],
 ['try',
  'ry ',
  'y s',
  ' so',
  'som',
  'ome',
  'met',
  'eth',
  'thi',
  'hin',
  'ing',
  'ng.']]

### Hashing

We consider languages with many characters that will make the number of bigrams and trigrams impossible to process. We will use the _hashing trick_ to reduce them, where we will gather $n$-grams into subsets using hash codes.

Each item will have this format:
`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

#### Description

Python has a built-in hashing function that returns a unique numerical signature for a given string

In [27]:
hash('a'), hash('ab'), hash('abc')

(-2487669937101553324, -8617692203368088064, -886363846305662418)

If we take the remainder (modulo) of a division by 5, we reduce the possible codes to: 0, 1, 2, 3, or 4

In [28]:
list(map(lambda x: x % 5, (hash('a'), hash('ab'), hash('abc'))))

[1, 1, 2]

#### Implementation

We set maximal numbers for our $n$-grams using these divisors

In [29]:
MAX_CHARS = 521
MAX_BIGRAMS = 1031
MAX_TRIGRAMS = 1031

Here strings have integer codes within the range [0, `MAX_CHARS`[

In [30]:
list(map(lambda x: x % MAX_CHARS, (hash('a'), hash('ab'), hash('abc'))))

[214, 220, 320]

Hash codes may vary across machines and Marcus Klang wrote this function to have reproducible codes

In [31]:
def reproducible_hash(string):
    """
    reproducible hash on any string
    
    Arguments:
       string: python string object
    
    Returns:
       signed int64
    """
    
    # We are using MD5 for speed not security.
    h = hashlib.md5(string.encode("utf-8"), usedforsecurity=False)
    return int.from_bytes(h.digest()[0:8], 'big', signed=True)

In [32]:
reproducible_hash('a')

919145239626757800

In [33]:
reproducible_hash('a') % MAX_CHARS

234

### Converting $n$-grams to hash codes
You will now convert the $n$-grams to hash codes


In [34]:
MAXES = [MAX_CHARS, MAX_BIGRAMS, MAX_TRIGRAMS]

Create a `hash_ngrams` function that creates a list of hash codes from a list of $n$-grams. As arguments, you will have the list of $n$-grams `[chars, bigrams, trigrams]` as well as the list of dividers (`MAXES`).

The output format will be a list of three lists:

`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

In [35]:
# Write your code
def hash_ngrams(ngrams, modulos):
    hash_codes = []
    for i in range(len(ngrams)):
        ngram_hash = []
        ngram = ngrams[i]
        modulo = modulos[i]
        for j in range(len(ngram)):
            ngram_hash.append(reproducible_hash(ngram[j]) % modulo)
        hash_codes.append(ngram_hash)
    return hash_codes

In [36]:
all_ngrams('try something.')

[['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.'],
 ['tr',
  'ry',
  'y ',
  ' s',
  'so',
  'om',
  'me',
  'et',
  'th',
  'hi',
  'in',
  'ng',
  'g.'],
 ['try',
  'ry ',
  'y s',
  ' so',
  'som',
  'ome',
  'met',
  'eth',
  'thi',
  'hin',
  'ing',
  'ng.']]

In [37]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

### Functions to Count Hash Codes

Write a function `calc_rel_freq(codes)` to count the codes. As in CLD3, you will return the relative frequencies.

This is just an application of `Counter` to a list of codes and then a division by the length.

The input is a list of codes and the output is a `Counter` object of relative frequencies.

In [38]:
# Write your code
def calc_rel_freq(codes):
    length = len(codes)
    cnt = Counter(codes)
    for key in cnt:
        cnt[key] = cnt[key] / length
    return cnt

In [39]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

In [40]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

[Counter({432: 0.14285714285714285,
          437: 0.07142857142857142,
          309: 0.07142857142857142,
          86: 0.07142857142857142,
          331: 0.07142857142857142,
          97: 0.07142857142857142,
          100: 0.07142857142857142,
          32: 0.07142857142857142,
          332: 0.07142857142857142,
          233: 0.07142857142857142,
          310: 0.07142857142857142,
          31: 0.07142857142857142,
          442: 0.07142857142857142}),
 Counter({6: 0.07692307692307693,
          765: 0.07692307692307693,
          224: 0.07692307692307693,
          203: 0.07692307692307693,
          557: 0.07692307692307693,
          176: 0.07692307692307693,
          590: 0.07692307692307693,
          711: 0.07692307692307693,
          527: 0.07692307692307693,
          757: 0.07692307692307693,
          919: 0.07692307692307693,
          57: 0.07692307692307693,
          685: 0.07692307692307693}),
 Counter({848: 0.08333333333333333,
          617: 0.08333333333333

### Merge the Dictionaries

In the results above, we have three counter objects with numerical keys (the hash codes). You will build one dictionary of them.

There is a key overlap and we must take care that a same hash code for the unigrams is not the same as in the bigrams. We will then shift the keys.

The keys range from:
1. Unigrams from 0 to 521, [0, MAX_CHARS[
2. Bigrams from 0 to 1031, [0, MAX_BIGRAMS[
3. Trigrams from 1 to 1031, [0, MAX_TRIGRAMS[

You will leave the unigrams keys as they are. You will shift the bigram keys by MAX_CHARS, and the trigram keys by MAX_CHARS + MAX_BIGRAMS. You can reuse the code below

In [41]:
MAX_SHIFT = []
for i in range(len(MAXES)):
    MAX_SHIFT += [sum(MAXES[:i])]

In [42]:
MAX_SHIFT

[0, 521, 1552]

Write a `shift_keys(dicts, MAX_SHIFT)` function that takes a list of dictionaries as input and the list of shifts and that a new unique dictionary, where the numerical keys have been shifted by the numbers in `MAX_SHIFT`

In [43]:
# Write your code here
def shift_keys(dicts, MAX_SHIFT):
    new_dict = {}
    for i in range(len(dicts)):
        dictionary = dicts[i]
        max_shift = MAX_SHIFT[i]
        for key in dictionary:
            new_dict[key + max_shift] = dictionary[key]
    return new_dict

In [44]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

[Counter({432: 0.14285714285714285,
          437: 0.07142857142857142,
          309: 0.07142857142857142,
          86: 0.07142857142857142,
          331: 0.07142857142857142,
          97: 0.07142857142857142,
          100: 0.07142857142857142,
          32: 0.07142857142857142,
          332: 0.07142857142857142,
          233: 0.07142857142857142,
          310: 0.07142857142857142,
          31: 0.07142857142857142,
          442: 0.07142857142857142}),
 Counter({6: 0.07692307692307693,
          765: 0.07692307692307693,
          224: 0.07692307692307693,
          203: 0.07692307692307693,
          557: 0.07692307692307693,
          176: 0.07692307692307693,
          590: 0.07692307692307693,
          711: 0.07692307692307693,
          527: 0.07692307692307693,
          757: 0.07692307692307693,
          919: 0.07692307692307693,
          57: 0.07692307692307693,
          685: 0.07692307692307693}),
 Counter({848: 0.08333333333333333,
          617: 0.08333333333333

In [45]:
shift_keys(list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES))), MAX_SHIFT)

{432: 0.14285714285714285,
 437: 0.07142857142857142,
 309: 0.07142857142857142,
 86: 0.07142857142857142,
 331: 0.07142857142857142,
 97: 0.07142857142857142,
 100: 0.07142857142857142,
 32: 0.07142857142857142,
 332: 0.07142857142857142,
 233: 0.07142857142857142,
 310: 0.07142857142857142,
 31: 0.07142857142857142,
 442: 0.07142857142857142,
 527: 0.07692307692307693,
 1286: 0.07692307692307693,
 745: 0.07692307692307693,
 724: 0.07692307692307693,
 1078: 0.07692307692307693,
 697: 0.07692307692307693,
 1111: 0.07692307692307693,
 1232: 0.07692307692307693,
 1048: 0.07692307692307693,
 1278: 0.07692307692307693,
 1440: 0.07692307692307693,
 578: 0.07692307692307693,
 1206: 0.07692307692307693,
 2400: 0.08333333333333333,
 2169: 0.08333333333333333,
 2020: 0.08333333333333333,
 2008: 0.08333333333333333,
 2425: 0.08333333333333333,
 2548: 0.08333333333333333,
 1839: 0.08333333333333333,
 1562: 0.08333333333333333,
 2369: 0.08333333333333333,
 2226: 0.08333333333333333,
 2512: 0.08333

Finally, we assemble all these utilities in a function

In [46]:
def build_freq_dict(sentence, MAXES=MAXES, MAX_SHIFT=MAX_SHIFT):
    hngrams = hash_ngrams(all_ngrams(sentence), MAXES)
    fhcodes = list(map(calc_rel_freq, hngrams))
    return shift_keys(fhcodes, MAX_SHIFT)

In [47]:
build_freq_dict('try something.')

{432: 0.14285714285714285,
 437: 0.07142857142857142,
 309: 0.07142857142857142,
 86: 0.07142857142857142,
 331: 0.07142857142857142,
 97: 0.07142857142857142,
 100: 0.07142857142857142,
 32: 0.07142857142857142,
 332: 0.07142857142857142,
 233: 0.07142857142857142,
 310: 0.07142857142857142,
 31: 0.07142857142857142,
 442: 0.07142857142857142,
 527: 0.07692307692307693,
 1286: 0.07692307692307693,
 745: 0.07692307692307693,
 724: 0.07692307692307693,
 1078: 0.07692307692307693,
 697: 0.07692307692307693,
 1111: 0.07692307692307693,
 1232: 0.07692307692307693,
 1048: 0.07692307692307693,
 1278: 0.07692307692307693,
 1440: 0.07692307692307693,
 578: 0.07692307692307693,
 1206: 0.07692307692307693,
 2400: 0.08333333333333333,
 2169: 0.08333333333333333,
 2020: 0.08333333333333333,
 2008: 0.08333333333333333,
 2425: 0.08333333333333333,
 2548: 0.08333333333333333,
 1839: 0.08333333333333333,
 1562: 0.08333333333333333,
 2369: 0.08333333333333333,
 2226: 0.08333333333333333,
 2512: 0.08333

## Converting the Dataset
We can now enrich the dataset with a numerical representation of the sentence. We use the utility functions and we call this new version: `dataset_num`

In [48]:
dataset[:2]

[('8638547', 'dan', 'Min far er for nylig vendt tilbage fra Japan.'),
 ('98775', 'jpn', '彼らがいつやってくるかわからない。')]

In [49]:
dataset_num = []
for datapoint in tqdm(dataset):
    dataset_num += [list(datapoint) + [build_freq_dict(datapoint[2])]]

  9%|▉         | 27464/300000 [00:05<00:44, 6058.31it/s]

100%|██████████| 300000/300000 [00:44<00:00, 6763.48it/s]


In [50]:
dataset_num[:2]

[['8638547',
  'dan',
  'Min far er for nylig vendt tilbage fra Japan.',
  {100: 0.022222222222222223,
   233: 0.06666666666666667,
   310: 0.08888888888888889,
   86: 0.17777777777777778,
   327: 0.06666666666666667,
   234: 0.1111111111111111,
   437: 0.08888888888888889,
   32: 0.06666666666666667,
   97: 0.022222222222222223,
   309: 0.022222222222222223,
   15: 0.044444444444444446,
   31: 0.044444444444444446,
   121: 0.022222222222222223,
   371: 0.022222222222222223,
   432: 0.044444444444444446,
   25: 0.022222222222222223,
   192: 0.022222222222222223,
   379: 0.022222222222222223,
   442: 0.022222222222222223,
   1294: 0.022727272727272728,
   1440: 0.022727272727272728,
   1222: 0.022727272727272728,
   1547: 0.06818181818181818,
   531: 0.022727272727272728,
   557: 0.022727272727272728,
   555: 0.09090909090909091,
   912: 0.022727272727272728,
   1066: 0.022727272727272728,
   819: 0.022727272727272728,
   1207: 0.022727272727272728,
   1540: 0.022727272727272728,
   101

## Programming: Building ${X}$

You will now build the ${X}$ matrix.

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

Note that we used characters and not codes to make it more legible.

To create the ${X}$ matrix, we need to transform the dictionaries of `dataset_num` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the hash code frequency dictionaries from `dataset_num` corresponding to its 3rd index;
2. Convert the list of dictionaries into an ${X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the unigrams only. Each item in this list will be a dictionary. You will call it `X_cat`

In [51]:
# Write your code here
X_cat = list(map(lambda x: x[3], dataset_num))

In [52]:
X_cat[0]

{100: 0.022222222222222223,
 233: 0.06666666666666667,
 310: 0.08888888888888889,
 86: 0.17777777777777778,
 327: 0.06666666666666667,
 234: 0.1111111111111111,
 437: 0.08888888888888889,
 32: 0.06666666666666667,
 97: 0.022222222222222223,
 309: 0.022222222222222223,
 15: 0.044444444444444446,
 31: 0.044444444444444446,
 121: 0.022222222222222223,
 371: 0.022222222222222223,
 432: 0.044444444444444446,
 25: 0.022222222222222223,
 192: 0.022222222222222223,
 379: 0.022222222222222223,
 442: 0.022222222222222223,
 1294: 0.022727272727272728,
 1440: 0.022727272727272728,
 1222: 0.022727272727272728,
 1547: 0.06818181818181818,
 531: 0.022727272727272728,
 557: 0.022727272727272728,
 555: 0.09090909090909091,
 912: 0.022727272727272728,
 1066: 0.022727272727272728,
 819: 0.022727272727272728,
 1207: 0.022727272727272728,
 1540: 0.022727272727272728,
 1010: 0.022727272727272728,
 1498: 0.022727272727272728,
 909: 0.022727272727272728,
 1029: 0.022727272727272728,
 1304: 0.02272727272727272

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html. You will set the `sparse` argument to False. Call the result `X`.

In [53]:
# Write your code here
v = DictVectorizer(sparse=False)
X = v.fit_transform(X_cat)

In [54]:
X.shape

(300000, 2583)

In [55]:
X[:5]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.11111111, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [56]:
# Write your code here
y_cat = list(map(lambda x: x[1], dataset_num))

In [57]:
y_cat[:5]

['dan', 'jpn', 'jpn', 'jpn', 'cmn']

Extract the set of language symbols and name it `y_symbols`. Then build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2idx`and `idx2lang`. Such a conversion is not necessary with sklearn. We do it because many other many machine-learning toolkits (keras or pytorch) require a numerical $\mathbf{y}$ vector and to learn how to carry out this conversion.

In [58]:
# Write your code here
y_symbols = set(y_cat)
y_symbols_list = list(y_symbols)
idx2lang = dict(enumerate(y_symbols_list))
lang2idx = {v: k for k, v in idx2lang.items()}

In [59]:
idx2lang

{0: 'swe', 1: 'dan', 2: 'fra', 3: 'cmn', 4: 'eng', 5: 'jpn'}

In [60]:
lang2idx

{'swe': 0, 'dan': 1, 'fra': 2, 'cmn': 3, 'eng': 4, 'jpn': 5}

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [61]:
# Write your code here
y = list(map(lambda x: lang2idx[x], y_cat))

In [62]:
y[:5]

[1, 5, 5, 5, 3]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call your classifier `clf`

In [63]:
# Write your code here
clf = MLPClassifier(hidden_layer_sizes=50 ,max_iter=5, verbose=True)

In [64]:
clf

MLPClassifier(hidden_layer_sizes=50, max_iter=5, verbose=True)

### Training and Validation Sets

You will now split the dataset into a training and validation sets

#### We split the dataset
We use a training set of 80% and a validation set of 20%

In [65]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

X_val

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Fitting the model

Fit the model on the training set

In [66]:
# Write your code here
model = clf.fit(X_train, y_train)

Iteration 1, loss = 0.28322136
Iteration 2, loss = 0.03695551
Iteration 3, loss = 0.02621839
Iteration 4, loss = 0.02178995
Iteration 5, loss = 0.01917231


/home/jonatan/anaconda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [67]:
# Write your code here
y_val_pred = model.predict(X_val)

In [68]:
y_val_pred[:20]

array([3, 3, 3, 0, 5, 0, 5, 5, 1, 4, 4, 0, 5, 3, 1, 1, 5, 4, 5, 4])

In [69]:
y_val[:20]

[3, 3, 3, 0, 5, 0, 5, 5, 1, 4, 4, 0, 5, 3, 1, 1, 5, 4, 5, 4]

#### Evaluating

Use the `accuracy_score()` function to evaluate your model on the validation set

In [70]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

0.9918

In [71]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         swe       0.98      0.98      0.98     10066
         dan       0.97      0.98      0.98      9963
         fra       1.00      1.00      1.00     10041
         cmn       1.00      1.00      1.00      9866
         eng       1.00      1.00      1.00     10059
         jpn       1.00      1.00      1.00     10005

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.9918
Macro F1 0.9918136393339484


### Confusion Matrix

In [72]:
confusion_matrix(y_val, y_val_pred)

array([[ 9815,   243,     5,     0,     3,     0],
       [  139,  9803,     1,     0,    20,     0],
       [    2,     7, 10022,     0,    10,     0],
       [    7,     6,     0,  9840,     2,    11],
       [    4,    18,     7,     0, 10030,     0],
       [    0,     1,     0,     5,     1,  9998]])

You may try to increase the number of iterations to improve the score. You may also try change the parameters of the multilayer percetron.

## Predict the language of a text

Now you will predict the languages of the strings below.

In [73]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

In [74]:
build_freq_dict('Salut les gars !')

{331: 0.1875,
 234: 0.125,
 15: 0.125,
 69: 0.0625,
 432: 0.0625,
 86: 0.1875,
 32: 0.0625,
 31: 0.0625,
 437: 0.0625,
 333: 0.0625,
 1078: 0.06666666666666667,
 640: 0.06666666666666667,
 582: 0.06666666666666667,
 1542: 0.06666666666666667,
 1492: 0.06666666666666667,
 900: 0.06666666666666667,
 739: 0.06666666666666667,
 1319: 0.06666666666666667,
 1238: 0.13333333333333333,
 982: 0.06666666666666667,
 1415: 0.06666666666666667,
 557: 0.06666666666666667,
 1161: 0.06666666666666667,
 1020: 0.06666666666666667,
 1803: 0.07142857142857142,
 1608: 0.07142857142857142,
 2199: 0.07142857142857142,
 2349: 0.07142857142857142,
 2284: 0.07142857142857142,
 1958: 0.07142857142857142,
 1720: 0.07142857142857142,
 1925: 0.07142857142857142,
 2370: 0.07142857142857142,
 2546: 0.07142857142857142,
 1752: 0.07142857142857142,
 1805: 0.07142857142857142,
 1670: 0.07142857142857142,
 2269: 0.07142857142857142}

Create features vectors from this list. Call this matrix `X_test`

In [75]:
# Write your code here
X_test = []
for i in docs:
    X_test.append(build_freq_dict(i))
X_2 = v.transform(X_test)

In [76]:
X_2.shape

(4, 2583)

And run the prediction that you will store in a variable called `pred_languages`

In [77]:
# Write your code here
pred_languages = list(map(lambda x: idx2lang[x], model.predict(X_2)))

In [78]:
pred_languages

['fra', 'swe', 'eng', 'dan']

## Building the Model with PyTorch
You will now recreate a PyTorch model with the same architecture as in sklearn.

### The Model
Create a model identical to the one you created with sklearn. Use the same activation function for the hidden layer and no activation in the last layer.

In [79]:
len(langs)

6

In [87]:
# Write your code here
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, len(langs))
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

        

In [88]:
input_dim = X.shape[1]
model = Model(input_dim)
model

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Write the loss `loss_fn` and optimizer `optimizer`. As optimizer, use the same as in sklearn. See here: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [89]:
# Write your code here. (The solution is given)
loss_fn = nn.CrossEntropyLoss()    # cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### The data loader

We convert the data to tensors

In [90]:
X_train = torch.Tensor(X_train)
y_train = torch.LongTensor(y_train)

X_val = torch.Tensor(X_val)
y_val = torch.LongTensor(y_val)

X_2 = torch.Tensor(X_2)

In [91]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [92]:
model.train()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Fit your network on your training set. Write a code similar to that seen during the lecture and use five epochs to start with.

In [93]:
# Write your code here
for epoch in range(5):
    loss_train = 0
    for X_batch, y_batch in dataloader:
        y_batch_pred = model(X_batch)
        loss = loss_fn(y_batch_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    print(loss_train)

726.6003119733941
149.3065166273591
120.19755977327532
102.37031644777517
90.48539036102903


Predict the validation set `X_val` in the form of logits. Call the result: `Y_val_pred_logits`

In [94]:
model.eval()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

In [97]:
# Write your code here
Y_val_pred_logits = model(X_val)

In [98]:
Y_val_pred_logits[:5]

tensor([[-11.6713,  -5.9205,  -8.7120,  15.3056, -15.4852,   2.3609],
        [-12.2138,  -6.6449,  -6.4395,  13.9440,  -9.4918,   1.6138],
        [ -1.3285, -15.5488, -10.5047,  13.8485,  -9.1206,   2.3544],
        [ 10.2528,  -2.0747, -13.8516,  -9.5763, -10.2802, -13.5591],
        [-10.7569,  -8.6204, -10.9065,  -3.5733,  -8.7299,  20.8561]],
       grad_fn=<SliceBackward0>)

Predict the validation set `X_val` in the form of probabilities. Use `torch.softmax()` for that and call the result: `Y_val_pred_proba`

In [100]:
# Write your code here
Y_val_pred_proba = F.softmax(Y_val_pred_logits, dim=-1)

In [101]:
Y_val_pred_proba[:4]

tensor([[1.9234e-12, 6.0478e-10, 3.7093e-11, 1.0000e+00, 4.2435e-14, 2.3888e-06],
        [4.3633e-12, 1.1438e-09, 1.4046e-09, 1.0000e+00, 6.6368e-11, 4.4163e-06],
        [2.5629e-07, 1.7097e-13, 2.6518e-11, 9.9999e-01, 1.0584e-10, 1.0190e-05],
        [1.0000e+00, 4.4282e-06, 3.4007e-11, 2.4452e-09, 1.2095e-09, 4.5560e-11]],
       grad_fn=<SliceBackward0>)

Extract the categories from the probabilities in `Y_val_pred_proba`. Use the `torch.argmax()` function. Call the result `y_val_pred`. Check that the prediction corresponds to the real values.

In [102]:
# Write your code here
y_val_pred = torch.argmax(Y_val_pred_proba, dim=-1)

In [103]:
y_val_pred[:20]

tensor([3, 3, 3, 0, 5, 0, 5, 5, 1, 4, 4, 0, 5, 3, 1, 1, 5, 4, 5, 4])

In [104]:
y_val[:20]

tensor([3, 3, 3, 0, 5, 0, 5, 5, 1, 4, 4, 0, 5, 3, 1, 1, 5, 4, 5, 4])

Print the evaluation

In [105]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         swe       0.99      0.98      0.98     10066
         dan       0.97      0.99      0.98      9963
         fra       1.00      1.00      1.00     10041
         cmn       1.00      1.00      1.00      9866
         eng       1.00      1.00      1.00     10059
         jpn       1.00      1.00      1.00     10005

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.9928333333333333
Macro F1 0.9928452805163838


Print the confusion matrix

In [106]:
confusion_matrix(y_val, y_val_pred)

array([[ 9819,   237,     5,     0,     5,     0],
       [  106,  9845,     1,     0,    11,     0],
       [    2,     7, 10027,     0,     5,     0],
       [    1,     2,     1,  9848,     2,    12],
       [    4,    19,     5,     0, 10031,     0],
       [    0,     0,     0,     5,     0, 10000]])

Predict your languages with PyTorch. Reuse `X_test` and call the result `Y_test_pred_proba`.

In [107]:
# Write your code here
Y_test_pred_proba = F.softmax(model(X_2), dim=-1)

In [108]:
Y_test_pred_proba

tensor([[3.4079e-06, 8.1188e-07, 9.9999e-01, 1.0700e-08, 5.0348e-06, 2.0433e-09],
        [9.9940e-01, 5.3867e-04, 4.1682e-06, 2.3007e-05, 2.9710e-05, 3.4907e-07],
        [5.7848e-05, 8.5325e-03, 5.7842e-04, 4.2217e-05, 9.9079e-01, 1.6197e-06],
        [2.8964e-04, 9.9970e-01, 2.3688e-06, 1.1637e-05, 1.1145e-07, 3.4991e-07]],
       grad_fn=<SoftmaxBackward0>)

From the probabilities, extract the predicted languages and map them to strings. Call the results `pred_languages_pytorch`.

In [118]:
# Write your code here
y_test_pred = torch.argmax(Y_test_pred_proba, dim=-1).tolist()
pred_languages_pytorch = list(map(lambda x: idx2lang[x], y_test_pred))

In [119]:
pred_languages_pytorch

['fra', 'swe', 'eng', 'dan']

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to:
   * Summarize CLD3 and outline its architecture
   * Identify the features used by CLD3
   * Describe your architecture and tell how it is different from CLD3
   * Include the feature matrix you computed manually
   * Outline the differences between sklearn and PyTorch

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 6, 2023.

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote in 2019 for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.